In [1]:
!nvidia-smi

Tue Apr 30 14:44:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:1A:00.0 Off |                    0 |
| N/A   42C    P0              59W / 300W |   4613MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [24]:
# !pip install -Uqqq pip --progress-bar off
!pip install -qqq peft==0.5.0 # this library gives us tools to use the LoRA technique for fine-tuning
# !pip install bitsandbytes # for quantization, which helps us load the model in 4 bits
# !pip install trl==0.7.1 # provides a trainer class that we'll use to fine-tune the model. trl stands for training transformers with reinforcement learning
# import peft
# import trl
# peft.__version__

ERROR: Ignored the following versions that require a different python version: 0.10.0 Requires-Python >=3.8.0; 0.4.0 Requires-Python >=3.8.0; 0.5.0 Requires-Python >=3.8.0; 0.6.0 Requires-Python >=3.8.0; 0.6.1 Requires-Python >=3.8.0; 0.6.2 Requires-Python >=3.8.0; 0.7.0 Requires-Python >=3.8.0; 0.7.1 Requires-Python >=3.8.0; 0.8.0 Requires-Python >=3.8.0; 0.8.1 Requires-Python >=3.8.0; 0.8.2 Requires-Python >=3.8.0; 0.9.0 Requires-Python >=3.8.0
ERROR: Could not find a version that satisfies the requirement peft==0.5.0 (from versions: 0.0.1, 0.0.2, 0.1.0, 0.2.0, 0.3.0)
ERROR: No matching distribution found for peft==0.5.0


In [102]:
# !pip install -Uqqq pip --progress-bar off
# !pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

ERROR: Ignored the following yanked versions: 4.14.0, 4.25.0
ERROR: Ignored the following versions that require a different python version: 4.31.0 Requires-Python >=3.8.0; 4.32.0 Requires-Python >=3.8.0; 4.32.1 Requires-Python >=3.8.0; 4.33.0 Requires-Python >=3.8.0; 4.33.1 Requires-Python >=3.8.0; 4.33.2 Requires-Python >=3.8.0; 4.33.3 Requires-Python >=3.8.0; 4.34.0 Requires-Python >=3.8.0; 4.34.1 Requires-Python >=3.8.0; 4.35.0 Requires-Python >=3.8.0; 4.35.1 Requires-Python >=3.8.0; 4.35.2 Requires-Python >=3.8.0; 4.36.0 Requires-Python >=3.8.0; 4.36.1 Requires-Python >=3.8.0; 4.36.2 Requires-Python >=3.8.0; 4.37.0 Requires-Python >=3.8.0; 4.37.1 Requires-Python >=3.8.0; 4.37.2 Requires-Python >=3.8.0; 4.38.0 Requires-Python >=3.8.0; 4.38.1 Requires-Python >=3.8.0; 4.38.2 Requires-Python >=3.8.0; 4.39.0 Requires-Python >=3.8.0; 4.39.1 Requires-Python >=3.8.0; 4.39.2 Requires-Python >=3.8.0; 4.39.3 Requires-Python >=3.8.0; 4.40.0 Requires-Python >=3.8.0; 4.40.1 Requires-Python >=3.8

In [111]:
# !pip install trl
# !pip install typing
# import trl
# from typing_extensions import Literal
python__version

In [1]:
import json
import re
from pprint import pprint
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import interpreter_login
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
# import trl
DEVICE = 'cuda:0' if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-hf" #"decapoda-research/llama-7b-hf" #  # meta model is a base model with no instruction tuned.

2024-04-30 16:11:01.455825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to /home/mmoha014/.cache/huggingface/token
Login successful


# Data Preprocessing

The dataset is a collection of conversations between a customer and a support agent over Twitter. The data is provided by Salesforce and is available on the HuggingFace datasets hub. The dataset contains 1099 conversations, split into 879 training , 110 validation and 110 testing samples.

In [3]:
dataset = load_dataset("Salesforce/dialogstudio", "TweetSumm")
dataset

/home/mmoha014/envs/subtab/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for Salesforce/dialogstudio contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Salesforce/dialogstudio
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

We are interested in two fields:
- Original dialog Info - the summary  of the conversation
- log - the conversation itself

The following functions put together the conversation text from the log field of the data point. It also cleans the text by removing URLs, mentions, and extra spaces. Here's an example conversation:

In [4]:
# def clean_text(text):
#     text = re.sub(r"http\S+", "", text)
#     text = re.sub(r"@[^\s]+", "", text)
#     text = re.sub(r"\s+", " ", text)
#     text = re.sub(r"\^[^ ]+", "", text)
#     return text

# def create_conversation_text(data_point):
#     text = ""
#     for item in data_point['log']:
#         user = clean_text(item['user utterance'])
#         text += f"user: {user.strip()}\n"
        
#         agent = clean_text(item['system response'])
#         text += f"agent: {agent.strip()}\n"
#     return text
        
        
# def generate_text(data_point):
#     summaries = json.loads(data_point['original dialog info'])['summaries']['abstractive_summaries']
#     summary = summaries[0]
#     summary = " ".join(summary)
#     conversation_text = create_conversation_text(data_point)
#     return {
#         "conversation":conversation_text,
#         "summary": summary,
#         "text": generate_training_prompt(conversation_text, summary)
#     }
def create_conversation_text(data_point):
    text = ""
    for item in data_point["log"]:
        user = clean_text(item["user utterance"])
        text += f"user: {user.strip()}\n"
 
        agent = clean_text(item["system response"])
        text += f"agent: {agent.strip()}\n"
 
    return text
 
 
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@[^\s]+", "", text)
    text = re.sub(r"\s+", " ", text)
    return re.sub(r"\^[^ ]+", "", text)

def generate_text(data_point):
    summaries = json.loads(data_point["original dialog info"])["summaries"][
        "abstractive_summaries"
    ]
    summary = summaries[0]
    summary = " ".join(summary)
 
    conversation_text = create_conversation_text(data_point)
    return {
        "conversation": conversation_text,
        "summary": summary,
        "text": generate_training_prompt(conversation_text, summary),
    }

### The prompt generation function
Here we use Alapaca-style prompt format

In [5]:
DEFAULT_SYSTEM_PROMPT = """
Below is a conversation between a human and an AI agent. Write a summary of the conversation.
""".strip()
 
 
def generate_training_prompt(
    conversation: str, summary: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT
) -> str:
    return f"""### Instruction: {system_prompt}
 
### Input:
{conversation.strip()}
 
### Response:
{summary}
""".strip()

We can now process the whole dataset using a helper function

In [6]:
def process_dataset(data:Dataset):
    return (
        data.shuffle(seed=42).map(generate_text).remove_columns(['original dialog id', 'new dialog id','original dialog info','log','prompt'])
    )

dataset['train'] = process_dataset(dataset['train'])
dataset['validation'] = process_dataset(dataset['validation'])
dataset

DatasetDict({
    train: Dataset({
        features: ['dialog index', 'conversation', 'summary', 'text'],
        num_rows: 879
    })
    validation: Dataset({
        features: ['dialog index', 'conversation', 'summary', 'text'],
        num_rows: 110
    })
    test: Dataset({
        features: ['original dialog id', 'new dialog id', 'dialog index', 'original dialog info', 'log', 'prompt'],
        num_rows: 110
    })
})

In [7]:
dataset['validation']['text'][2]

"### Instruction: Below is a conversation between a human and an AI agent. Write a summary of the conversation.\n \n### Input:\nuser: My subscription expired and it won't allow me to change my card number. And the system says my email address doesn't exist... Now I can't pay online nor in the Xbox store. I also bought 3 different codes from Walmart and they didn't work. #HELPNEEDED\nagent: We are not equipped to handle account/billing issues here on Twitter, for security reasons. In this case, let's have 1  you reach out to the live chat team in your region: for more help with your account! 2\nuser: I’d rather talk to someone over the phone if possible. Is there a number I could call and preferred time\nagent: The link we sent will also have an option for the phone if available at that time and in that region.\nuser: Yes I am aware. But in order to get the number I have to sign in. Which for some reason I can not. This is my only method of communication\nagent: You should not need to s

# Model fine tuning
We want to use the base 7b verion of Llama 2 model. We will load it using the <u>bitsandbytes</u> library to load the model in 4 bits precision. 

In [8]:
def create_model_and_tokenizer():
    # for the 4bit quntization, we're using normalized float(nf) with 4 bits
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb4bit_compute_dtype=torch.bfloat16,
    )
    # we use "use_safetensors" option to enable safe tensors format loading
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )
    
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'right'
    return model, tokenizer

In [9]:
model, tokenizer = create_model_and_tokenizer()
model.config.use_cache=False

Unused kwargs: ['bnb4bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The transformers library integrates nicely with different quantization libraries. We can check the quantization configuration of the model as following:

In [10]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 '_load_in_8bit': False,
 '_load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float32',
 'bnb_4bit_quant_storage': 'uint8',
 'load_in_4bit': True,
 'load_in_8bit': False}

### QLoRA configuration

In [11]:
lora_r = 16  # set the rank of update matrices to 16
lora_alpha = 32 # the weight matrix is scaled by (lora_alpha/r)
lora_dropout = 0.1
lora_target_modules = ["q_proj",]#,"up_proj","o_proj","k_proj","down_proj","gate_proj","v_proj",]

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    target_modules=lora_target_modules,
    bias='none',
    task_type='CAUSAL_LM'
)

# Training
We'll use Tensorboard to monitor the training process

In [12]:
OUTPUT_DIR = 'experiments'
%load_ext tensorboard
%tensorboard --logdir experiments/runs

Most of the settings are self-explanatory. We're using:

    * paged_adamw_32bit optimizer, which is a memory-efficient version of AdamW
    * cosine learning rate scheduler
    * The group_by_length option to group samples of roughly the same length together. This can help with training stability.


In [16]:
# Setting up the training parameters
training_arguments = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_8bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy='steps',
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir='none',#OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type='cosine',
    seed=42,
)

 Here the trl library is used for trainer class. It's a wrapper around the transformers library. Additional to the standard training class, we'll pas in the <u>peft_config</u> and the <u>dataset_text_field</u> options. The latter is required to tell the trainer which filed to use for the training the prompt

In [17]:
trainer = SFTTrainer(model=model, train_dataset=dataset['train'], eval_dataset=dataset['validation'],
                     peft_config=peft_config, dataset_text_field='text',
                     max_seq_length=4096, tokenizer=tokenizer, args=training_arguments)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
# Let's start the training
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 106.00 MiB (GPU 0; 15.77 GiB total capacity; 10.33 GiB already allocated; 61.56 MiB free; 10.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# save the model
trainer.save_model()

# Merge the QLoRA adapter with Llama2
This will result in a single model that we can use for inference

In [ ]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(OUTPUT_DIR,low_cpu_mem_usage=True,)

merged_model = model.merge_and_unload()
merged_model.save_pretrained('merged_model', sae_serialization=True)
tokenizer.save_pretrained('merged_model')

# Another Fine-Tuning

In [55]:
# https://www.youtube.com/watch?v=XpoKB3usmKc
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
import transformers


!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='auto', trust_remote_code=False, revision='main')

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME,use_fast=True) #use_fast

model.train() # model in training mode (dropout modules are activated)
model.gradient_checkpointing_enable() # enable gradient check pointing
model = prepare_model_for_kbit_training(model) # enable quantized training

# Setting up LoRA
# config = LoraConfig(r=8, lora_alpha=32, target_modules=['q_proj'], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
# 29% trainable parameters
config = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj","up_proj","o_proj","k_proj","down_proj","gate_proj","v_proj",], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
model = get_peft_model(model, config)
# LoRA trainable parameter count
model.print_trainable_parameters()
